# 电网数学模型emo

In [8]:
import numpy as np
from scipy.optimize import minimize

# 时间段数量（假设24小时）
T = 24
Δt = 1  # 时间间隔，假设为1小时

# 参数初始化
p_e_s = np.random.uniform(0.1, 0.5, T)  # EMO售出的电能的价格,在t=24的前提下，这个代码相当于生成了长度为24的数组，每个元素是0.1到0.5之间的一个随机数
L_e = np.random.uniform(10, 50, T)      # 用能侧的实际电负荷
P_BT_chr = np.random.uniform(5, 20, T)  # EMO向ESO的售电功率
P_es = np.random.uniform(5, 25, T)      # EMO从EGO的购电功率
p_e_b = np.random.uniform(0.1, 0.5, T)  # EMO从EGO的购电价格
p_g_s = np.random.uniform(0.1, 0.5, T)  # EMO向电网的售电价格
p_g_b = np.random.uniform(0.1, 0.5, T)  # EMO向电网的购电价格
P_g_s_max = 50                          # 售电功率上限
P_g_b_max = 50                          # 购电功率上限

# 目标函数
def objective(p_e_s):
    C_sell1 = p_e_s * L_e * Δt
    C_sell2 = p_e_s * P_BT_chr * Δt
    C_grid = (np.maximum(L_e - P_es, 0) * p_g_s + np.minimum(L_e - P_es, 0) * p_g_b + p_e_b * P_es) * Δt
    E_EMO = np.sum(C_sell1 + C_sell2 - C_grid)
    return -E_EMO  # 最大化E_EMO

# 约束条件
def constraints(p_e_s):
    avg_p_e_s = np.mean(p_e_s)
    cons = []

    # 电价约束
    cons.append(p_g_b - p_e_s)  # p_g_b^t < p_e_s^t
    cons.append(p_e_s - p_g_s)  # p_e_s^t < p_g_s^t
    cons.append(p_g_b - p_e_b)  # p_g_b^t < p_e_b^t
    cons.append(p_e_b - p_g_s)  # p_e_b^t < p_g_s^t

    # 平均售电价格约束
    cons.append(np.sum(p_e_s) - T * avg_p_e_s)

    return np.concatenate(cons)

# # 初始猜测值
# initial_guess = np.random.uniform(0.1, 0.5, T)

# # 约束条件字典
# constraints_dict = [{'type': 'ineq', 'fun': lambda p_e_s: con} for con in constraints(initial_guess)]

# # 求解优化问题
# result = minimize(objective, initial_guess, constraints=constraints_dict, bounds=[(0, 1)] * T)

# # 输出结果
# if result.success:
#     optimal_p_e_s = result.x
#     print(f"Optimal selling prices: {optimal_p_e_s}")
# else:
#     print("Optimization failed. Check the constraints and initial guesses.")


# 供能ego

In [7]:
import numpy as np
from scipy.optimize import minimize

# 时间段数量
T = 24

# 参数初始化
P_WT = np.random.uniform(10, 20, T)       # WT的输出功率
P_PV = np.random.uniform(5, 15, T)        # PV的输出功率
P_GT = np.random.uniform(0, 10, T)        # CCHP系统输出的电功率
P_ISAC = np.random.uniform(1, 5, T)       # ISAC的耗电功率
C_sell = np.random.uniform(0.5, 1.5, T)   # 售氢收益
F_cEGO = np.random.uniform(0.1, 0.3, T)   # 碳交易成本
a_e, b_e, c_e = 0.01, 0.1, 1              # CCHP的成本系数
c_Q, c_T = 5, 10                          # GT的启、停成本系数
Δt = 1                                    # 时间间隔，假设为1小时

# 目标函数
def objective(x):
    P_GT, I_GT_Q, I_GT_T = x[:T], x[T:2*T], x[2*T:]
    C_cchp = a_e * P_GT**2 + b_e * P_GT + c_e
    C_QT = c_Q * I_GT_Q + c_T * I_GT_T
    E_EGO = np.sum(C_sell - C_cchp - F_cEGO - C_QT)
    return -E_EGO  # 负号表示最大化

# 约束条件
def constraints(x):
    P_GT, I_GT_Q, I_GT_T = x[:T], x[T:2*T], x[2*T:]
    P_es = P_WT + P_PV + P_GT - P_ISAC
    cons = []
    cons.append(P_es)   # P_es >= 0
    return np.concatenate([c.flatten() for c in cons])

# # 初始猜测值
# initial_guess = np.concatenate([np.random.uniform(0, 10, T), np.zeros(T), np.zeros(T)])

# # 约束条件字典
# constraints_dict = [{'type': 'ineq', 'fun': lambda x, i=i: constraints(x)[i]} for i in range(len(constraints(initial_guess)))]

# # 求解优化问题
# result = minimize(objective, initial_guess, constraints=constraints_dict, bounds=[(0, 10)] * T + [(0, 1)] * (2 * T))

# # 输出结果
# if result.success:
#     optimal_solution = result.x
#     optimal_P_GT = optimal_solution[:T]
#     print(f"Optimal GT power output: {optimal_P_GT}")
# else:
#     print("Optimization failed. Check the constraints and initial guesses.")


In [5]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel('15.xlsx')

# 只处理前24000行
df_subset = df.iloc[:23201]

# 用上一行的值填充缺失值
df_subset.fillna(method='ffill', inplace=True)

# 将处理后的前24000行和未处理的剩余行合并
df.iloc[:23201] = df_subset

# 将修改后的DataFrame写回Excel文件
df.to_excel('1_filled.xlsx', index=False)

print("前23201行的缺失值已填充并保存到1_filled.xlsx")



前30401行的缺失值已填充并保存到1_filled.xlsx


只考虑厂区的博弈模型：
数据使用：预测模型的发电数据、24h随即电价

电解槽模型以及状态
类定义：Electrolyzer 类用于表示电解槽及其状态。
初始化参数：包括电解槽的额定功率、启动时的电能损失系数和最低运行负荷率。
状态方法：stop、start 和 work 方法用于设置电解槽的不同状态。
状态转换：state_transition 方法根据二进制变量s、w、l来转换状态。
状态约束：single_state_constraint 方法用于确保在任意时刻电解槽只能处于一种状态。
更新状态：update 方法用于更新电解槽状态并应用约束条件。
通过这种方式，你可以模拟电解槽在不同状态下的行为，并进行进一步的优化和分析。

In [1]:
import numpy as np

class Electrolyzer:
    def __init__(self, P_max, beta_I, beta_EL_min):
        self.P_max = P_max  # 额定功率，MW
        self.beta_I = beta_I  # 启动状态时的电能损失系数
        self.beta_EL_min = beta_EL_min  # 最低运行负荷率
        
        self.S = 0  # 停机状态
        self.L = 0  # 启动状态
        self.W = 0  # 工作状态
        self.W_prev = 0  # 前一时刻工作状态
        
        self.P_H2 = 0  # 有效制氢功率，MW
        self.E_EL = 0  # 负载功率，MW
    
    def stop(self):
        self.S = 1
        self.L = 0
        self.W = 0
        self.P_H2 = 0
        self.E_EL = 0
    
    def start(self):
        self.S = 0
        self.L = 1
        self.W = 0
        self.P_H2 = 0
        self.E_EL = self.beta_I * self.P_max
    
    def work(self, E_EL):
        if E_EL < self.beta_EL_min * self.P_max or E_EL > self.P_max:
            raise ValueError("Load power out of bounds")
        
        self.S = 0
        self.L = 0
        self.W = 1
        self.P_H2 = E_EL
        self.E_EL = E_EL
    
    def state_transition(self, s, w, l):
        if s == 1 and w == 1:
            self.stop()
        elif w == 1 and l == 1:
            self.work(self.E_EL)
        elif l == 1 and s == 1:
            self.start()
        else:
            self.S = s
            self.W = w
            self.L = l
    
    def single_state_constraint(self):
        if self.S + self.L + self.W != 1:
            raise ValueError("State constraint violated")
    
    def update(self):
        self.single_state_constraint()
        if self.S == 1 and self.W_prev == 1:
            self.stop()
        elif self.L == 1 and self.S == 1:
            self.start()
        elif self.W == 1 and self.L == 1:
            self.work(self.E_EL)
        self.W_prev = self.W

# 参数设置
P_max = 100  # 额定功率，MW
beta_I = 0.2  # 启动状态时的电能损失系数
beta_EL_min = 0.1  # 最低运行负荷率

# 创建电解槽实例
electrolyzer = Electrolyzer(P_max, beta_I, beta_EL_min)

# 模拟电解槽状态转换
state_sequence = [(1, 1, 0), (0, 1, 1), (0, 0, 1)]  # (s, w, l)

for state in state_sequence:
    s, w, l = state
    electrolyzer.state_transition(s, w, l)
    electrolyzer.update()
    print(f"State: S={electrolyzer.S}, W={electrolyzer.W}, L={electrolyzer.L}")
    print(f"Power: P_H2={electrolyzer.P_H2} MW, E_EL={electrolyzer.E_EL} MW")


State: S=1, W=0, L=0
Power: P_H2=0 MW, E_EL=0 MW


ValueError: Load power out of bounds

In [2]:
# gai
import numpy as np

class Electrolyzer:
    def __init__(self, P_max, beta_I, beta_EL_min):
        self.P_max = P_max  # 额定功率，MW
        self.beta_I = beta_I  # 启动状态时的电能损失系数
        self.beta_EL_min = beta_EL_min  # 最低运行负荷率
        
        self.S = 0  # 停机状态
        self.L = 0  # 启动状态
        self.W = 0  # 工作状态
        self.W_prev = 0  # 前一时刻工作状态
        
        self.P_H2 = 0  # 有效制氢功率，MW
        self.E_EL = 0  # 负载功率，MW
    
    def stop(self):
        self.S = 1
        self.L = 0
        self.W = 0
        self.P_H2 = 0
        self.E_EL = 0
    
    def start(self):
        self.S = 0
        self.L = 1
        self.W = 0
        self.P_H2 = 0
        self.E_EL = self.beta_I * self.P_max
    
    def work(self, E_EL):
        if E_EL < self.beta_EL_min * self.P_max or E_EL > self.P_max:
            raise ValueError("Load power out of bounds")
        
        self.S = 0
        self.L = 0
        self.W = 1
        self.P_H2 = E_EL
        self.E_EL = E_EL
    
    def state_transition(self, s, w, l, E_EL=None):
        if s == 1 and w == 1:
            self.stop()
        elif w == 1 and l == 1:
            if E_EL is not None:
                self.work(E_EL)
            else:
                self.work(self.P_max)  # 默认使用最大功率
        elif l == 1 and s == 1:
            self.start()
        else:
            self.S = s
            self.W = w
            self.L = l
    
    def single_state_constraint(self):
        if self.S + self.L + self.W != 1:
            raise ValueError("State constraint violated")
    
    def update(self):
        self.single_state_constraint()
        if self.S == 1 and self.W_prev == 1:
            self.stop()
        elif self.L == 1 and self.S == 1:
            self.start()
        elif self.W == 1 and self.L == 1:
            self.work(self.E_EL)
        self.W_prev = self.W

# 参数设置
P_max = 100  # 额定功率，MW
beta_I = 0.2  # 启动状态时的电能损失系数
beta_EL_min = 0.1  # 最低运行负荷率

# 创建电解槽实例
electrolyzer = Electrolyzer(P_max, beta_I, beta_EL_min)

# 模拟电解槽状态转换
state_sequence = [
    (1, 1, 0),  # 停机
    (0, 1, 1),  # 启动
    (0, 0, 1, 50),  # 工作，50 MW
    (1, 1, 0)   # 停机
]  # (s, w, l, E_EL)

for state in state_sequence:
    s, w, l, *E_EL = state
    E_EL = E_EL[0] if E_EL else None
    electrolyzer.state_transition(s, w, l, E_EL)
    electrolyzer.update()
    print(f"State: S={electrolyzer.S}, W={electrolyzer.W}, L={electrolyzer.L}")
    print(f"Power: P_H2={electrolyzer.P_H2} MW, E_EL={electrolyzer.E_EL} MW")


State: S=1, W=0, L=0
Power: P_H2=0 MW, E_EL=0 MW
State: S=0, W=1, L=0
Power: P_H2=100 MW, E_EL=100 MW
State: S=0, W=0, L=1
Power: P_H2=100 MW, E_EL=100 MW
State: S=1, W=0, L=0
Power: P_H2=0 MW, E_EL=0 MW


In [ ]:
# gai2


import numpy as np

class Electrolyzer:
    def __init__(self, P_max, beta_I, beta_EL_min, switch_cost, lifetime_cost):
        self.P_max = P_max  # 额定功率，MW
        self.beta_I = beta_I  # 启动状态时的电能损失系数
        self.beta_EL_min = beta_EL_min  # 最低运行负荷率
        
        self.switch_cost = switch_cost  # 状态切换成本
        self.lifetime_cost = lifetime_cost  # 寿命成本
        
        self.S = 0  # 停机状态
        self.L = 0  # 启动状态
        self.W = 0  # 工作状态
        self.W_prev = 0  # 前一时刻工作状态
        
        self.P_H2 = 0  # 有效制氢功率，MW
        self.E_EL = 0  # 负载功率，MW
        
        self.lifetime = 0  # 电解槽使用寿命
        self.total_cost = 0  # 总成本
    
    def stop(self):
        self.S = 1
        self.L = 0
        self.W = 0
        self.P_H2 = 0
        self.E_EL = 0
        self.total_cost += self.switch_cost
    
    def start(self):
        self.S = 0
        self.L = 1
        self.W = 0
        self.P_H2 = 0
        self.E_EL = self.beta_I * self.P_max
        self.total_cost += self.switch_cost
    
    def work(self, E_EL):
        if E_EL < self.beta_EL_min * self.P_max or E_EL > self.P_max:
            raise ValueError("Load power out of bounds")
        
        self.S = 0
        self.L = 0
        self.W = 1
        self.P_H2 = E_EL
        self.E_EL = E_EL
        self.total_cost += self.lifetime_cost
        self.lifetime += 1
    
    def state_transition(self, s, w, l, E_EL=None):
        if s == 1 and w == 1:
            self.stop()
        elif w == 1 and l == 1:
            if E_EL is not None:
                self.work(E_EL)
            else:
                self.work(self.P_max)  # 默认使用最大功率
        elif l == 1 and s == 1:
            self.start()
        else:
            self.S = s
            self.W = w
            self.L = l
    
    def single_state_constraint(self):
        if self.S + self.L + self.W != 1:
            raise ValueError("State constraint violated")
    
    def update(self):
        self.single_state_constraint()
        if self.S == 1 and self.W_prev == 1:
            self.stop()
        elif self.L == 1 and self.S == 1:
            self.start()
        elif self.W == 1 and self.L == 1:
            self.work(self.E_EL)
        self.W_prev = self.W

# 参数设置
P_max = 100  # 额定功率，MW
beta_I = 0.2  # 启动状态时的电能损失系数
beta_EL_min = 0.1  # 最低运行负荷率
switch_cost = 10  # 状态切换成本，假设为10单位
lifetime_cost = 5  # 每次工作状态下的寿命成本，假设为5单位








# 创建电解槽组
num_electrolyzers = 8
electrolyzers = [Electrolyzer(P_max, beta_I, beta_EL_min, switch_cost, lifetime_cost) for _ in range(num_electrolyzers)]

# 初始状态设定
initial_states = [(0, 0, 1, 50) for _ in range(4)] + [(0, 1, 1) for _ in range(2)] + [(1, 1, 0) for _ in range(2)]

for i, state in enumerate(initial_states):
    s, w, l, *E_EL = state
    E_EL = E_EL[0] if E_EL else None
    electrolyzers[i].state_transition(s, w, l, E_EL)
    electrolyzers[i].update()

# 输出初始状态
for i, electrolyzer in enumerate(electrolyzers):
    print(f"Electrolyzer {i+1}:")
    print(f"  State: S={electrolyzer.S}, W={electrolyzer.W}, L={electrolyzer.L}")
    print(f"  Power: P_H2={electrolyzer.P_H2} MW, E_EL={electrolyzer.E_EL} MW")
    print(f"  Total Cost: {electrolyzer.total_cost}, Lifetime: {electrolyzer.lifetime}")

# 模拟电解槽组的状态转换
state_sequences = [
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 50), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 70), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 90), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 60), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 80), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 50), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 70), (1, 1, 0)],
    [(1, 1, 0), (0, 1, 1), (0, 0, 1, 90), (1, 1, 0)]
]  # 每个电解槽的状态序列 (s, w, l, E_EL)

for i, state_sequence in enumerate(state_sequences):
    print(f"Electrolyzer {i+1}:")
    for state in state_sequence:
        s, w, l, *E_EL = state
        E_EL = E_EL[0] if E_EL else None
        electrolyzers[i].state_transition(s, w, l, E_EL)
        electrolyzers[i].update()
        print(f"  State: S={electrolyzers[i].S}, W={electrolyzers[i].W}, L={electrolyzers[i].L}")
        print(f"  Power: P_H2={electrolyzers[i].P_H2} MW, E_EL={electrolyzers[i].E_EL} MW")
        print(f"  Total Cost: {electrolyzers[i].total_cost}, Lifetime: {electrolyzers[i].lifetime}")


代码说明
类定义：Electrolyzer 类添加了 switch_cost 和 lifetime_cost 属性，用于表示状态切换成本和寿命成本。
方法修改：
stop 方法和 start 方法中增加了状态切换成本的累加。
work 方法中增加了寿命成本的累加和使用寿命的增加。
初始状态设定：通过 initial_states 设置电解槽组的初始状态。
模拟状态转换：通过 state_sequences 模拟每个电解槽的状态转换，并输出状态和成本信息。
这个模型能够帮助你分析电解槽组在不同状态下的运行成本和寿命管理，从而优化整体的运行策略